In [309]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import category_encoders as ce

In [310]:
df = pd.read_csv('hotels.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [311]:
columns = df.columns.to_list()
num_cols = list()
cat_cols = list()

for column in columns:
    if df[column].dtype == 'O':
        cat_cols.append(column)
    else:
        num_cols.append(column)

len(cat_cols)
len(num_cols)

9

In [312]:
feature_df = pd.DataFrame()
df['hotel_name'].nunique()
feature_df['review_date'] = pd.to_datetime(df['review_date'])
feature_df['review_date'].max()

def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

# feature_df['tags'] = ''
# feature_df['tags'] = df['tags'].apply(rev_func)
feature_df['hotel_name'] = df['hotel_name']

# feature_df = feature_df.explode('tags')

In [313]:
# feature_df['tags'].sort_values(ascending = False)
# feature_df.groupby('tags')['hotel_name'].count().sort_values(ascending=False)

In [314]:
def get_country(address):
    country = address.split(' ')[-2:]
    country = str(country).replace(',', '').replace("['", "").replace("']", "").replace("'", "")
    if country == 'United Kingdom':
        return 'United Kingdom'
    if country == 'Paris France':
        return 'France'
    if country == 'Amsterdam Netherlands':
        return 'Amsterdam'
    if country == 'Milan Italy':
        return 'Italy'
    if country == 'Vienna Austria':
        return 'Austria'
    if country == 'Barcelona Spain':
        return 'Spain'
    else:
        return 'other'
    
df['country'] = ''
df['country'] = df['hotel_address'].apply(get_country)


def if_capital(country, address):
    if country == 'United Kingdom':
        if 'London' in address:
            return 'yes'
        else:
            return 'no'
    if country == 'France':
        if 'Paris' in address:
            return 'yes'
        else:
            return 'no'
    if country == 'Amsterdam':
        if 'Netherlands' in address:
            return 'yes'
        else:
            return 'no'
    if country == 'Italy':
        if 'Milan' in address:
            return 'yes'
        else:
            return 'no'
    if country == 'Austria':
        if 'Vienna' in address:
            return 'yes'
        else:
            return 'no'
    if country == 'Spain':
        if 'Barcelona' in address:
            return 'yes'
        else:
            return 'no'
        
feature_df['hotel_address'] = df['hotel_address']


url = 'https://www.belastingdienst.nl/wps/wcm/connect/bldcontenten/belastingdienst/individuals/tax-regulations/tax_treaties/list-of-eu-countries/'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
eu_countries = page.find('article').find_all('li')
eu_countries = list(eu_countries)

for i in range(len(eu_countries)):
    eu_countries[i] = (str(eu_countries[i])).replace('<li>','').replace('</li>', '').replace('\xa0', ' ').replace('*', '').replace("'", "")
eu_countries


['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'The Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'the Åland Islands',
 'the Azores',
 'the Canary Islands',
 'French Guiana',
 'Gibraltar',
 'Guadeloupe',
 'Madeira',
 'Martinique',
 'Mayotte',
 'Réunion',
 'Saint Martin']

In [315]:
feature_df['review_date'] = feature_df['review_date'].dt.year
df['reviewer_nationality'].nunique()
feature_df
feature_df = feature_df.drop('hotel_address', axis = 1)
feature_df = feature_df.drop('hotel_name', axis = 1)

In [316]:
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) 
type_bin = bin_encoder.fit_transform(df['reviewer_nationality'])
feature_df = pd.concat([feature_df, type_bin], axis=1)

In [317]:

feature_df['days_since_review'] = df['days_since_review'].apply(lambda x: str(x.split(' ')[:1]).replace("'", '').replace("'", '').replace('[', '').replace(']', ''))
feature_df['days_since_review'] = feature_df['days_since_review'].astype('float64')
                                                         

In [318]:
feature_df['additional_number_of_scoring'] = ''
feature_df['average_score'] = ''
feature_df['review_total_negative_word_counts'] = ''
feature_df['total_number_of_reviews'] = ''
feature_df['review_total_positive_word_counts'] = ''
feature_df['total_number_of_reviews_reviewer_has_given'] = ''
feature_df['reviewer_score'] = ''

feature_df['additional_number_of_scoring'] = df['additional_number_of_scoring']
feature_df['average_score'] = round(df['average_score'])
feature_df['review_total_negative_word_counts'] = df['review_total_negative_word_counts']
feature_df['total_number_of_reviews'] = df['total_number_of_reviews']
feature_df['review_total_positive_word_counts'] = df['review_total_positive_word_counts']
feature_df['total_number_of_reviews_reviewer_has_given'] = df['total_number_of_reviews_reviewer_has_given']
feature_df['reviewer_score'] = round(df['reviewer_score'])

In [319]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=['country'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(df['country'])
feature_df = pd.concat([feature_df, type_bin], axis=1)


In [338]:
df

hotel_address                                 Lancaster Gate Westminster Borough London W2 3...
additional_number_of_scoring                                                                452
review_date                                                                            7/4/2016
average_score                                                                               7.6
hotel_name                                                                Corus Hotel Hyde Park
reviewer_nationality                                                            United Kingdom 
negative_review                                The room we were given was smaller than I exp...
review_total_negative_word_counts                                                            48
total_number_of_reviews                                                                    2562
positive_review                                Enjoyed our breakfast and the surroundings in...
review_total_positive_word_counts       

In [320]:
df['if_capital'] = df.apply(lambda row: if_capital(row['country'], row['hotel_address']), axis = 1)

encodery = ce.OneHotEncoder(cols=['if_capital'], use_cat_names=True) # указываем столбец для кодирования
type_biny = encodery.fit_transform(df['if_capital'])
feature_df = pd.concat([feature_df, type_biny], axis=1)

In [321]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = feature_df.drop(['reviewer_score'], axis = 1)  
y = feature_df['reviewer_score'] 

In [322]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [323]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [324]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [325]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14585933822409763


In [330]:
# feature_df[feature_df['if_capital'].isnull() == True]
# feature_df['if_capital'].info()
feature_df[feature_df['if_capital_yes'] == 1]

,review_date,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7,days_since_review,...,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,country_United Kingdom,country_France,country_Amsterdam,country_Italy,country_Austria,country_Spain,if_capital_yes
0,2016,0,0,0,0,0,0,0,1,531.0,...,4,7,10.0,1,0,0,0,0,0,1
1,2017,0,0,0,0,0,0,0,1,203.0,...,2,14,6.0,1,0,0,0,0,0,1
2,2016,0,0,0,0,0,0,1,0,289.0,...,0,14,8.0,0,1,0,0,0,0,1
3,2015,0,0,0,0,0,0,0,1,681.0,...,11,8,10.0,0,1,0,0,0,0,1
4,2016,0,0,0,0,0,0,1,1,516.0,...,20,10,10.0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386798,2017,0,0,0,0,0,1,0,0,106.0,...,10,10,9.0,1,0,0,0,0,0,1
386799,2017,0,0,1,0,1,0,1,1,171.0,...,93,1,9.0,0,0,0,0,1,0,1
386800,2016,0,0,0,0,1,1,1,1,543.0,...,9,21,8.0,1,0,0,0,0,0,1
386801,2017,0,0,0,0,1,0,0,0,74.0,...,6,28,9.0,1,0,0,0,0,0,1


In [327]:
# df['tags'] = df['tags'].apply(lambda x: x.replace("' ", "'"))
# df['tags'] = df['tags'].apply(lambda x: x.replace(" '", "'"))
# df['tags'] = df['tags'].apply(lambda x: x.replace("[", ""))
# df['tags'] = df['tags'].apply(lambda x: x.replace("]", ""))
# df['tags'] = df['tags'].apply(lambda x: x.replace("'", ""))

# tags_dict = dict()

# def unique_tags(data):
#     # data = str(data)
#     for elem in data:
#         if elem not in tags_dict:
#             tags_dict[elem] = 1
#         else:
#             tags_dict[elem] += 1
#     return tags_dict
    
# df['tags']
# md = sorted(unique_tags(df['tags']).items())